In [1]:
# taken from singan implementaion from repo given below
# https://github.com/FriedRonaldo/SinGAN
#
class Generator(nn.Module):
    def __init__(self, img_size_min, num_scale, size_list, bs, heads, d_model, dim1, dim2, dropout = 0.4, scale_factor=4/3):
        super(Generator, self).__init__()
        self.img_size_min = img_size_min
        self.scale_factor = scale_factor
        self.num_scale = num_scale
        self.nf = 32
        self.current_scale = 0
        self.d_model = d_model
        self.h = heads
        self.dim1 = dim1
        self.dim2 =dim2
        self.dropout = dropout
        self.nu_feat =d_model # int(d_model/self.h*2)
        self.bs = bs

        self.size_list = size_list #[int(self.img_size_min * scale_factor**i) for i in range(num_scale + 1)]
        print(self.size_list)
        self.MHA= MultiHeadAttention(bs, heads, d_model, dim1, dim2, dropout )
        self.sub_generators = nn.ModuleList()

        first_generator = nn.ModuleList()

        first_generator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1),
                                             nn.BatchNorm2d(self.nf),
                                             nn.LeakyReLU(2e-1)))
        for _ in range(3):
            first_generator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1),
                                                 nn.BatchNorm2d(self.nf),
                                                 nn.LeakyReLU(2e-1)))

        first_generator.append(nn.Sequential(nn.Conv2d(self.nf, 3, 3, 1),
                                             nn.Tanh()))

        first_generator = nn.Sequential(*first_generator)

        self.sub_generators.append(first_generator)

    def forward(self, batch, img=None):
        x_list = []
        #print(batch[0].shape)
        dim2 = dim1 = batch[0].shape[-1]
        #self.MHA= MultiHeadAttention(self.bs, self.h, self.d_model,  dim1,  dim2, self.dropout )
        #x_first = self.sub_generators[0](self.MHA(batch[0]))
        #print(x_first.size())
        x_first = self.sub_generators[0](batch[0])
        x_list.append(x_first)

        if img is not None:
            x_inter = img
        else:
            x_inter = x_first

        for i in range(1, self.current_scale + 1):
            x_inter = F.interpolate(x_inter, (self.size_list[i], self.size_list[i]), mode='bilinear', align_corners=True)
            #print(x_inter.size())
            x_prev = x_inter
            x_inter = F.pad(x_inter, [5, 5, 5, 5], value=0)
            #print(self.current_scale)
            dim2 = dim1 = batch[i].shape[-1]
            """
            self.MHA= MultiHeadAttention(self.bs, self.h, self.d_model,  dim1,  dim2, self.dropout )
            x_inter = x_inter + self.MHA(batch[i])
            """
            x_inter = x_inter + (batch[i])
            x_inter = self.sub_generators[i](x_inter) + x_prev
            x_list.append(x_inter)
        #print(self.size_list)
        return x_list

    def progress(self):
        self.current_scale += 1

        if self.current_scale % 4 == 0:
            self.nf *= 2

        tmp_generator = nn.ModuleList()
        tmp_generator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1),
                                           nn.BatchNorm2d(self.nf),
                                           nn.LeakyReLU(2e-1)))

        for _ in range(3):
            tmp_generator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1),
                                               nn.BatchNorm2d(self.nf),
                                               nn.LeakyReLU(2e-1)))

        tmp_generator.append(nn.Sequential(nn.Conv2d(self.nf, 3, 3, 1),
                                           nn.Tanh()))

        tmp_generator = nn.Sequential(*tmp_generator)

        if self.current_scale % 4 != 0:
            prev_generator = self.sub_generators[-1]

            # Initialize layers via copy
            if self.current_scale >= 1:
                tmp_generator.load_state_dict(prev_generator.state_dict())

        self.sub_generators.append(tmp_generator)
        print("GENERATOR PROGRESSION DONE")

NameError: name 'nn' is not defined

In [ ]:
# taken from singan implementaion from repo given below
# https://github.com/FriedRonaldo/SinGAN
#


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.nf = 32
        self.current_scale = 0

        self.sub_discriminators = nn.ModuleList()

        first_discriminator = nn.ModuleList()

        first_discriminator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1, 1),
                                             nn.LeakyReLU(2e-1)))
        for _ in range(3):
            first_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1, 1),
                                                 nn.BatchNorm2d(self.nf),
                                                 nn.LeakyReLU(2e-1)))

        first_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, 1, 3, 1, 1)))

        first_discriminator = nn.Sequential(*first_discriminator)

        self.sub_discriminators.append(first_discriminator)

    def forward(self, x):
        out = self.sub_discriminators[self.current_scale](x)
        return out

    def progress(self):
        self.current_scale += 1
        # Lower scale discriminators are not used in later ... replace append to assign?
        if self.current_scale % 4 == 0:
            self.nf *= 2

        tmp_discriminator = nn.ModuleList()
        tmp_discriminator.append(nn.Sequential(nn.Conv2d(3, self.nf, 3, 1, 1),
                                               nn.LeakyReLU(2e-1)))

        for _ in range(3):
            tmp_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, self.nf, 3, 1, 1),
                                                   nn.BatchNorm2d(self.nf),
                                                   nn.LeakyReLU(2e-1)))

        tmp_discriminator.append(nn.Sequential(nn.Conv2d(self.nf, 1, 3, 1, 1)))

        tmp_discriminator = nn.Sequential(*tmp_discriminator)

        if self.current_scale % 4 != 0:
            prev_discriminator = self.sub_discriminators[-1]

            # Initialize layers via copy
            if self.current_scale >= 1:
                tmp_discriminator.load_state_dict(prev_discriminator.state_dict())

        self.sub_discriminators.append(tmp_discriminator)
        print("DISCRIMINATOR PROGRESSION DONE")